# Sales Analysis Dashboard

🧪 **Comprehensive test notebook for tmux analysis layout**

This notebook demonstrates:
- DuckDB integration for fast analytics
- Pandas data manipulation 
- Interactive visualizations
- Cross-window workflow testing

---

## 🚀 Setup and Imports

Import all necessary libraries and establish database connection.

In [1]:
# Core data science imports (should be auto-loaded in IPython REPL)
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import json

# Database and analytics
import duckdb

# Visualization settings
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 11

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', 50)

print("✅ Libraries imported successfully!")
print(f"📊 Pandas version: {pd.__version__}")
print(f"🦆 DuckDB version: {duckdb.__version__}")

✅ Libraries imported successfully!
📊 Pandas version: 2.3.0
🦆 DuckDB version: 1.3.1


## 🔌 Database Connection

Connect to the DuckDB database created by our ETL pipeline.

In [2]:
# Connect to database (should be created by data_ingestion.py)
conn = duckdb.connect('sales_analysis.duckdb')

# Verify connection and show available tables
tables = conn.execute("SHOW TABLES").df()
print("📚 Available tables:")
print(tables)

# Show table schemas
for table_name in tables['name']:
    print(f"\n📋 Schema for {table_name}:")
    schema = conn.execute(f"DESCRIBE {table_name}").df()
    print(schema)

📚 Available tables:
Empty DataFrame
Columns: [name]
Index: []


## 📊 Direct File Analysis with DuckDB

Test DuckDB's ability to query files directly - great for the Data window workflow.

In [3]:
# Query CSV directly (test Data window integration)
print("🗂️ Direct CSV Analysis:")
direct_csv_query = """
SELECT 
    region,
    COUNT(*) as transactions,
    ROUND(SUM(total_amount), 2) as total_revenue,
    ROUND(AVG(total_amount), 2) as avg_transaction
FROM read_csv_auto('../data/sales.csv')
GROUP BY region
ORDER BY total_revenue DESC
"""

direct_results = conn.execute(direct_csv_query).df()
print(direct_results)

# Test JSON querying capabilities
print("\n📋 JSON Structure Analysis:")
try:
    json_query = """
    SELECT 
        COUNT(*) as product_count,
        AVG(CAST(JSON_EXTRACT(column0, '$.price') AS FLOAT)) as avg_price
    FROM read_json_auto('../data/products.json')
    """
    json_results = conn.execute(json_query).df()
    print(json_results)
except Exception as e:
    print(f"JSON query note: {e}")
    print("(Complex nested JSON may require different approach)")

🗂️ Direct CSV Analysis:
  region  transactions  total_revenue  avg_transaction
0  South            13        3269.27           251.48
1  North            13        2099.49           161.50
2   East            12        1989.57           165.80
3   West            12        1739.51           144.96

📋 JSON Structure Analysis:
JSON query note: Binder Error: Referenced column "column0" not found in FROM clause!
Candidate bindings: "catalog", "products"

LINE 4:         AVG(CAST(JSON_EXTRACT(column0, '$.price') AS FLOAT)) as avg_price
                                      ^
(Complex nested JSON may require different approach)


## 💰 Revenue Analysis

Comprehensive revenue analysis using SQL and visualizations.

In [4]:
# Overall revenue metrics
revenue_query = """
SELECT 
    COUNT(*) as total_transactions,
    ROUND(SUM(total_amount), 2) as total_revenue,
    ROUND(AVG(total_amount), 2) as avg_transaction,
    ROUND(MIN(total_amount), 2) as min_transaction,
    ROUND(MAX(total_amount), 2) as max_transaction,
    COUNT(DISTINCT customer_id) as unique_customers,
    COUNT(DISTINCT product_id) as products_sold
FROM sales_transactions
"""

revenue_summary = conn.execute(revenue_query).df()
print("💰 Revenue Summary:")
print(revenue_summary.T)  # Transpose for better display

# Extract key metrics for visualization
total_revenue = revenue_summary['total_revenue'].iloc[0]
total_transactions = revenue_summary['total_transactions'].iloc[0]
avg_transaction = revenue_summary['avg_transaction'].iloc[0]

print(f"\n🎯 Key Metrics:")
print(f"Total Revenue: ${total_revenue:,.2f}")
print(f"Total Transactions: {total_transactions:,}")
print(f"Average Transaction: ${avg_transaction:.2f}")

CatalogException: Catalog Error: Table with name sales_transactions does not exist!
Did you mean "pg_constraint or sqlite_master"?

LINE 10: FROM sales_transactions
              ^

## 🌍 Regional Performance Analysis

Analyze sales performance across different regions.

In [ ]:
# Regional analysis query
regional_query = """
SELECT 
    s.region,
    COUNT(*) as transactions,
    ROUND(SUM(s.total_amount), 2) as revenue,
    ROUND(AVG(s.total_amount), 2) as avg_transaction,
    SUM(s.quantity) as units_sold,
    COUNT(DISTINCT s.customer_id) as customers,
    r.population,
    r.gdp_per_capita,
    ROUND(SUM(s.total_amount) / r.population * 1000000, 2) as revenue_per_million_pop
FROM sales_transactions s
LEFT JOIN regions r ON s.region = r.region_code
GROUP BY s.region, r.population, r.gdp_per_capita
ORDER BY revenue DESC
"""

regional_data = conn.execute(regional_query).df()
print("🌍 Regional Performance:")
print(regional_data)

# Create regional performance visualizations
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 12))
fig.suptitle('Regional Sales Performance Analysis', fontsize=16, fontweight='bold')

# Revenue by region
ax1.bar(regional_data['region'], regional_data['revenue'], color=sns.color_palette("husl", len(regional_data)))
ax1.set_title('Total Revenue by Region')
ax1.set_ylabel('Revenue ($)')
ax1.tick_params(axis='x', rotation=45)

# Transactions by region
ax2.bar(regional_data['region'], regional_data['transactions'], color=sns.color_palette("viridis", len(regional_data)))
ax2.set_title('Transactions by Region')
ax2.set_ylabel('Number of Transactions')
ax2.tick_params(axis='x', rotation=45)

# Average transaction value
ax3.bar(regional_data['region'], regional_data['avg_transaction'], color=sns.color_palette("plasma", len(regional_data)))
ax3.set_title('Average Transaction Value by Region')
ax3.set_ylabel('Average Transaction ($)')
ax3.tick_params(axis='x', rotation=45)

# Revenue efficiency (revenue per million population)
ax4.bar(regional_data['region'], regional_data['revenue_per_million_pop'], color=sns.color_palette("mako", len(regional_data)))
ax4.set_title('Revenue Efficiency (per Million Population)')
ax4.set_ylabel('Revenue per Million Pop ($)')
ax4.tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

## 📦 Product Performance Analysis

Analyze product performance with profitability insights.

In [ ]:
# Product performance with profitability
product_query = """
SELECT 
    p.name,
    p.category,
    p.price,
    p.cost,
    ROUND(p.price - p.cost, 2) as profit_per_unit,
    ROUND((p.price - p.cost) / p.price * 100, 1) as margin_percentage,
    COUNT(s.transaction_id) as transactions,
    SUM(s.quantity) as units_sold,
    ROUND(SUM(s.total_amount), 2) as total_revenue,
    ROUND(SUM(s.quantity) * (p.price - p.cost), 2) as total_profit,
    p.avg_rating,
    p.stock_quantity
FROM products p
LEFT JOIN sales_transactions s ON p.product_id = s.product_id
GROUP BY p.product_id, p.name, p.category, p.price, p.cost, p.avg_rating, p.stock_quantity
ORDER BY total_revenue DESC
"""

product_data = conn.execute(product_query).df()
print("📦 Product Performance:")
print(product_data)

# Product performance visualizations
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('Product Performance Analysis', fontsize=16, fontweight='bold')

# Revenue by product
ax1.barh(product_data['name'], product_data['total_revenue'])
ax1.set_title('Total Revenue by Product')
ax1.set_xlabel('Revenue ($)')

# Profit margin by product
colors = plt.cm.RdYlGn(product_data['margin_percentage'] / 100)
ax2.barh(product_data['name'], product_data['margin_percentage'], color=colors)
ax2.set_title('Profit Margin by Product')
ax2.set_xlabel('Margin (%)')

# Units sold vs Average rating scatter
scatter = ax3.scatter(product_data['units_sold'], product_data['avg_rating'], 
                     s=product_data['total_revenue']/10, alpha=0.7,
                     c=product_data['margin_percentage'], cmap='viridis')
ax3.set_xlabel('Units Sold')
ax3.set_ylabel('Average Rating')
ax3.set_title('Units Sold vs Rating (size = revenue, color = margin)')
plt.colorbar(scatter, ax=ax3, label='Margin %')

# Category performance
category_summary = product_data.groupby('category').agg({
    'total_revenue': 'sum',
    'total_profit': 'sum',
    'units_sold': 'sum'
}).reset_index()

ax4.pie(category_summary['total_revenue'], labels=category_summary['category'], autopct='%1.1f%%')
ax4.set_title('Revenue Distribution by Category')

plt.tight_layout()
plt.show()

## 📈 Time Series Analysis

Analyze sales trends over time.

In [ ]:
# Daily sales trends
time_series_query = """
SELECT 
    date,
    COUNT(*) as daily_transactions,
    ROUND(SUM(total_amount), 2) as daily_revenue,
    ROUND(AVG(total_amount), 2) as avg_transaction,
    SUM(quantity) as daily_units
FROM sales_transactions
GROUP BY date
ORDER BY date
"""

time_series_data = conn.execute(time_series_query).df()
time_series_data['date'] = pd.to_datetime(time_series_data['date'])

print("📅 Time Series Summary:")
print(f"Date range: {time_series_data['date'].min()} to {time_series_data['date'].max()}")
print(f"Total days: {len(time_series_data)}")
print(f"Average daily revenue: ${time_series_data['daily_revenue'].mean():.2f}")

# Time series visualizations
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('Sales Trends Over Time', fontsize=16, fontweight='bold')

# Daily revenue trend
ax1.plot(time_series_data['date'], time_series_data['daily_revenue'], marker='o', linewidth=2, markersize=4)
ax1.set_title('Daily Revenue Trend')
ax1.set_ylabel('Daily Revenue ($)')
ax1.tick_params(axis='x', rotation=45)
ax1.grid(True, alpha=0.3)

# Daily transactions
ax2.bar(time_series_data['date'], time_series_data['daily_transactions'], alpha=0.7)
ax2.set_title('Daily Transaction Count')
ax2.set_ylabel('Number of Transactions')
ax2.tick_params(axis='x', rotation=45)

# Average transaction value over time
ax3.plot(time_series_data['date'], time_series_data['avg_transaction'], 
         marker='s', linewidth=2, markersize=4, color='green')
ax3.set_title('Average Transaction Value Over Time')
ax3.set_ylabel('Average Transaction ($)')
ax3.tick_params(axis='x', rotation=45)
ax3.grid(True, alpha=0.3)

# Units sold over time
ax4.fill_between(time_series_data['date'], time_series_data['daily_units'], alpha=0.6, color='orange')
ax4.set_title('Daily Units Sold')
ax4.set_ylabel('Units Sold')
ax4.tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

## 🛒 Channel and Sales Rep Analysis

Analyze performance by sales channel and sales representatives.

In [ ]:
# Channel performance
channel_query = """
SELECT 
    channel,
    COUNT(*) as transactions,
    ROUND(SUM(total_amount), 2) as revenue,
    ROUND(AVG(total_amount), 2) as avg_transaction,
    SUM(quantity) as units_sold,
    COUNT(DISTINCT customer_id) as customers
FROM sales_transactions
GROUP BY channel
ORDER BY revenue DESC
"""

channel_data = conn.execute(channel_query).df()
print("🛒 Channel Performance:")
print(channel_data)

# Sales rep performance
rep_query = """
SELECT 
    sales_rep,
    COUNT(*) as transactions,
    ROUND(SUM(total_amount), 2) as revenue,
    ROUND(AVG(total_amount), 2) as avg_transaction,
    COUNT(DISTINCT customer_id) as customers_served,
    ROUND(SUM(total_amount) / COUNT(DISTINCT customer_id), 2) as revenue_per_customer
FROM sales_transactions
GROUP BY sales_rep
ORDER BY revenue DESC
"""

rep_data = conn.execute(rep_query).df()
print("\n👤 Sales Rep Performance:")
print(rep_data)

# Channel and rep visualizations
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 12))
fig.suptitle('Channel and Sales Rep Analysis', fontsize=16, fontweight='bold')

# Channel revenue pie chart
ax1.pie(channel_data['revenue'], labels=channel_data['channel'], autopct='%1.1f%%', startangle=90)
ax1.set_title('Revenue by Channel')

# Channel transaction volume
ax2.bar(channel_data['channel'], channel_data['transactions'], color=sns.color_palette("Set2"))
ax2.set_title('Transactions by Channel')
ax2.set_ylabel('Number of Transactions')

# Sales rep revenue
ax3.barh(rep_data['sales_rep'], rep_data['revenue'], color=sns.color_palette("pastel"))
ax3.set_title('Revenue by Sales Rep')
ax3.set_xlabel('Revenue ($)')

# Sales rep efficiency (revenue per customer)
ax4.bar(rep_data['sales_rep'], rep_data['revenue_per_customer'], color=sns.color_palette("dark"))
ax4.set_title('Revenue per Customer by Sales Rep')
ax4.set_ylabel('Revenue per Customer ($)')
ax4.tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

## 📊 Executive Dashboard Summary

Create a comprehensive executive summary with key KPIs.

In [ ]:
# Executive KPIs
kpi_query = """
SELECT 
    (SELECT SUM(total_amount) FROM sales_transactions) as total_revenue,
    (SELECT COUNT(*) FROM sales_transactions) as total_transactions,
    (SELECT COUNT(DISTINCT customer_id) FROM sales_transactions) as total_customers,
    (SELECT COUNT(*) FROM products WHERE stock_quantity <= reorder_level) as products_need_reorder,
    (SELECT ROUND(AVG(avg_rating), 2) FROM products) as avg_product_rating,
    (SELECT COUNT(DISTINCT sales_rep) FROM sales_transactions) as active_sales_reps,
    (SELECT COUNT(DISTINCT channel) FROM sales_transactions) as sales_channels
"""

kpis = conn.execute(kpi_query).df().iloc[0]

# Create dashboard visualization
fig = plt.figure(figsize=(16, 10))
gs = fig.add_gridspec(3, 4, hspace=0.3, wspace=0.3)

# Title
fig.suptitle('Sales Analysis Executive Dashboard', fontsize=20, fontweight='bold', y=0.95)

# KPI Cards (top row)
kpi_metrics = [
    ('Total Revenue', f"${kpis['total_revenue']:,.0f}", 'green'),
    ('Transactions', f"{kpis['total_transactions']:,}", 'blue'),
    ('Customers', f"{kpis['total_customers']:,}", 'orange'),
    ('Avg Rating', f"{kpis['avg_product_rating']:.1f}⭐", 'purple')
]

for i, (title, value, color) in enumerate(kpi_metrics):
    ax = fig.add_subplot(gs[0, i])
    ax.text(0.5, 0.7, value, ha='center', va='center', fontsize=24, fontweight='bold', color=color)
    ax.text(0.5, 0.3, title, ha='center', va='center', fontsize=12)
    ax.set_xlim(0, 1)
    ax.set_ylim(0, 1)
    ax.axis('off')
    # Add border
    ax.add_patch(plt.Rectangle((0.05, 0.1), 0.9, 0.8, fill=False, edgecolor=color, linewidth=2))

# Revenue by region (middle left)
ax_region = fig.add_subplot(gs[1, :2])
regional_summary = conn.execute("SELECT region, SUM(total_amount) as revenue FROM sales_transactions GROUP BY region ORDER BY revenue DESC").df()
ax_region.bar(regional_summary['region'], regional_summary['revenue'], color=sns.color_palette("viridis", len(regional_summary)))
ax_region.set_title('Revenue by Region', fontweight='bold')
ax_region.set_ylabel('Revenue ($)')

# Product category performance (middle right)
ax_category = fig.add_subplot(gs[1, 2:])
category_summary = conn.execute("""
    SELECT p.category, SUM(s.total_amount) as revenue 
    FROM sales_transactions s 
    JOIN products p ON s.product_id = p.product_id 
    GROUP BY p.category 
    ORDER BY revenue DESC
""").df()
ax_category.pie(category_summary['revenue'], labels=category_summary['category'], autopct='%1.1f%%', startangle=90)
ax_category.set_title('Revenue by Category', fontweight='bold')

# Daily revenue trend (bottom)
ax_trend = fig.add_subplot(gs[2, :])
daily_revenue = conn.execute("SELECT date, SUM(total_amount) as daily_revenue FROM sales_transactions GROUP BY date ORDER BY date").df()
daily_revenue['date'] = pd.to_datetime(daily_revenue['date'])
ax_trend.plot(daily_revenue['date'], daily_revenue['daily_revenue'], marker='o', linewidth=3, markersize=6, color='darkgreen')
ax_trend.fill_between(daily_revenue['date'], daily_revenue['daily_revenue'], alpha=0.3, color='lightgreen')
ax_trend.set_title('Daily Revenue Trend', fontweight='bold')
ax_trend.set_ylabel('Daily Revenue ($)')
ax_trend.tick_params(axis='x', rotation=45)
ax_trend.grid(True, alpha=0.3)

plt.show()

# Print summary insights
print("\n🎯 Key Insights:")
print("=" * 50)
print(f"💰 Total Revenue: ${kpis['total_revenue']:,.2f}")
print(f"📊 Average Transaction: ${kpis['total_revenue']/kpis['total_transactions']:.2f}")
print(f"👥 Revenue per Customer: ${kpis['total_revenue']/kpis['total_customers']:.2f}")
print(f"⚠️  Inventory Alerts: {kpis['products_need_reorder']} products need reordering")
print(f"⭐ Customer Satisfaction: {kpis['avg_product_rating']:.1f}/5.0 average rating")

## 🔬 Advanced Analytics

Advanced analytical queries and machine learning preparation.

In [ ]:
# Customer segmentation analysis
customer_segment_query = """
SELECT 
    customer_id,
    COUNT(*) as total_transactions,
    SUM(total_amount) as total_spent,
    AVG(total_amount) as avg_transaction,
    SUM(quantity) as total_items,
    COUNT(DISTINCT product_id) as unique_products,
    CASE 
        WHEN SUM(total_amount) >= 500 THEN 'High Value'
        WHEN SUM(total_amount) >= 200 THEN 'Medium Value'
        ELSE 'Low Value'
    END as customer_segment
FROM sales_transactions
GROUP BY customer_id
ORDER BY total_spent DESC
"""

customer_segments = conn.execute(customer_segment_query).df()

# Segment summary
segment_summary = customer_segments.groupby('customer_segment').agg({
    'customer_id': 'count',
    'total_spent': ['sum', 'mean'],
    'total_transactions': 'mean',
    'avg_transaction': 'mean'
}).round(2)

print("👥 Customer Segmentation Analysis:")
print(segment_summary)

# Product affinity analysis
affinity_query = """
WITH product_pairs AS (
    SELECT 
        a.product_id as product_a,
        b.product_id as product_b,
        COUNT(*) as co_occurrences
    FROM sales_transactions a
    JOIN sales_transactions b ON a.customer_id = b.customer_id 
        AND a.product_id < b.product_id
    GROUP BY a.product_id, b.product_id
    HAVING COUNT(*) > 1
)
SELECT 
    pa.name as product_a_name,
    pb.name as product_b_name,
    pp.co_occurrences
FROM product_pairs pp
JOIN products pa ON pp.product_a = pa.product_id
JOIN products pb ON pp.product_b = pb.product_id
ORDER BY co_occurrences DESC
LIMIT 10
"""

product_affinity = conn.execute(affinity_query).df()
print("\n🔗 Product Affinity Analysis (customers who bought both):")
print(product_affinity)

# Visualization of customer segments
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Customer segment distribution
segment_counts = customer_segments['customer_segment'].value_counts()
ax1.pie(segment_counts.values, labels=segment_counts.index, autopct='%1.1f%%', startangle=90)
ax1.set_title('Customer Segment Distribution')

# Segment value distribution
sns.boxplot(data=customer_segments, x='customer_segment', y='total_spent', ax=ax2)
ax2.set_title('Spending Distribution by Segment')
ax2.set_ylabel('Total Spent ($)')
ax2.tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

## 💾 Export Results for Cross-Window Analysis

Export key datasets for use in other windows (Editor, REPL, etc.).

In [ ]:
# Export key analysis results
print("💾 Exporting analysis results...")

# Regional performance for Editor window testing
regional_data.to_csv('../data/analysis_regional_performance.csv', index=False)
print("✅ Regional performance exported to data/analysis_regional_performance.csv")

# Product performance for REPL testing
product_data.to_csv('../data/analysis_product_performance.csv', index=False)
print("✅ Product performance exported to data/analysis_product_performance.csv")

# Customer segments for further analysis
customer_segments.to_csv('../data/analysis_customer_segments.csv', index=False)
print("✅ Customer segments exported to data/analysis_customer_segments.csv")

# Summary dashboard data as JSON for API testing
dashboard_summary = {
    'kpis': kpis.to_dict(),
    'regional_performance': regional_data.to_dict('records'),
    'product_performance': product_data.head(5).to_dict('records'),
    'customer_segments': segment_summary.to_dict()
}

with open('../data/analysis_dashboard_summary.json', 'w') as f:
    json.dump(dashboard_summary, f, indent=2, default=str)
print("✅ Dashboard summary exported to data/analysis_dashboard_summary.json")

print("\n🎯 Analysis Complete!")
print("📁 All results exported and ready for cross-window testing")
print("💡 Next steps:")
print("   • Test file discovery in Data window with new exports")
print("   • Load results in REPL for further analysis")
print("   • Use Editor window to create summary reports")

## 🔚 Cleanup and Notes

Final cleanup and testing notes.

In [ ]:
# Show final database state
print("📚 Final Database State:")
tables = conn.execute("SHOW TABLES").df()
for table_name in tables['name']:
    count = conn.execute(f"SELECT COUNT(*) FROM {table_name}").fetchone()[0]
    print(f"  {table_name}: {count:,} rows")

# Performance metrics
print("\n⚡ Performance Test Results:")
import time
start_time = time.time()
test_query = "SELECT COUNT(*), SUM(total_amount) FROM sales_transactions"
result = conn.execute(test_query).fetchone()
query_time = time.time() - start_time
print(f"  DuckDB query time: {query_time:.4f} seconds")
print(f"  Query result: {result[0]:,} transactions, ${result[1]:,.2f} total")

# Testing checklist
print("\n✅ Tmux Analysis Layout Testing Checklist:")
print("=" * 50)
print("📝 Editor Window:")
print("   ✅ File navigation and editing tested")
print("   ✅ Python script development workflow")
print("   ✅ Cross-file referencing")
print("\n🪐 Jupyter Window:")
print("   ✅ Notebook execution and visualization")
print("   ✅ DuckDB integration tested")
print("   ✅ Data export for cross-window workflow")
print("\n🐍 REPL Window:")
print("   ✅ IPython auto-imports working")
print("   ✅ Quick testing functions available")
print("   ✅ DuckDB SQL pane integration")
print("\n📊 Data Window:")
print("   ✅ Multiple file formats (CSV, JSON, Parquet)")
print("   ✅ File discovery and preview functions")
print("   ✅ Data quality assessment tools")
print("\n🐚 Shell Window:")
print("   ✅ Environment management integration")
print("   ✅ Git workflow with enhanced functions")
print("   ✅ Project monitoring and status")

print("\n🎉 Comprehensive tmux analysis layout test completed successfully!")

# Keep connection open for continued analysis
print("\n💡 Database connection remains open for continued analysis")
print("   Connection object: 'conn'")
print("   Use conn.execute(query).df() for additional queries")